#**LAB ASSINGMENT 9 - NLP**
Harshvardhan Aditya 
E20CSE365

In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [ ]:
text = open('/content/dataset_hindi_para.txt').read().lower()
print('Corpus Length:', len(text))

corpus length: 314


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
WORD_LENGTH = 100
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['यह', 'आप', 'व', 'भ', 'न', 'न', 'व', 'षय', 'पर', 'ह', 'द', 'न', 'ब', 'ध', 'पढ़', 'और', 'स', 'ख', 'सकत', 'ह', 'ज', 'स', 'क', 'ल', 'और', 'क', 'ल', 'ज', 'व', 'द', 'य', 'र', 'थ', 'य', 'क', 'प', 'छ', 'ज', 'त', 'ह', 'यह', 'आप', 'व', 'भ', 'न', 'न', 'व', 'षय', 'पर', 'ह', 'द', 'न', 'ब', 'ध', 'न', 'ब', 'ध', 'ल', 'खन', 'ह', 'ज', 'सभ', 'छ', 'त', 'र', 'एव', 'आम', 'न', 'गर', 'क', 'क', 'ल', 'ए', 'उपय', 'ग', 'ह', 'इस', 'व', 'बस', 'इट', 'पर', 'म', 'न', 'व', 'यवस', 'थ', 'त', 'र', 'प', 'स', 'सभ', 'प', 'रक', 'र', 'क', 'न', 'ब', 'ध', 'क', 'स']
ग


In [ ]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

In [ ]:
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
print(X[0][0])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False]


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 3.7523 - accuracy: 0.1250 - val_loss: 5.0777 - val_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 99ms/step - loss: 3.1016 - accuracy: 0.1250 - val_loss: 3.9013 - val_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 101ms/step - loss: 3.5698 - accuracy: 0.1875 - val_loss: 6.2244 - val_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 111ms/step - loss: 3.1641 - accuracy: 0.0625 - val_loss: 6.3079 - val_accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 96ms/step - loss: 3.1912 - accuracy: 0.1875 - val_loss: 5.7160 - val_accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 0s 100ms/step - loss: 2.7959 - accuracy: 0.0625 - val_loss: 6.7646 - val_accuracy: 0.0000e+00
Epoch 7/20
1/1 [==============================] - 0s 111ms/step - loss: 2.6276 - accuracy: 0.2500 - val_loss: 6.7064 - val_accuracy: 0.0000e+

In [ ]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x

In [ ]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [ ]:
q =  "यहाँ आप विभिन्न विषयों पर हिंदी"
print("Correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("The Sequence: ",seq)
print("The next possible words: ", predict_completions(seq, 5))

correct sentence:  यहाँ आप विभिन्न विषयों पर हिंदी
Sequence:  यह आप व भ न
यह
आप
व
भ
न
next possible words:  ['ह', 'य', 'द', 'यह', 'त']
